In [1]:
!pip install pyecharts==0.5.11

In [2]:
import numpy as np
import pandas as pd
import re
import os

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
import joblib
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split

import datetime
import json
import plotly.graph_objects as go
import matplotlib
import matplotlib.pyplot as plt
from pylab import mpl
#from pyecharts.charts import Geo
#from pyecharts.charts import Pie , Page , WordCloud
'''
from pyecharts.globals import ThemeType
from pyecharts.globals import SymbolType
import pyecharts.options as opts
'''
import seaborn as sns
from datetime import datetime
import warnings

sns.set_style(
    # 图形风格
    "whitegrid", 
    # 网格默认不显示
    {'axes.grid' : False})

sns.set_context(rc={
    # 标题大小
    'axes.titlesize': 25,
    # 轴的上标题大小
    'axes.labelsize': 20,
    # x轴刻度文字大小
    'xtick.labelsize': 15, 
    # y轴刻度文字大小
    'ytick.labelsize': 15})

plt.rcParams['font.sans-serif']='SimHei'
plt.rcParams['axes.unicode_minus'] = False  # 负号正常显示

warnings.filterwarnings('ignore')

In [4]:
#需要pyecharts版本在0.5.11
v_data = pd.read_csv("2021VAERSDATA-utf8.csv",encoding="utf-8")
v_text = v_data['SYMPTOM_TEXT'].tolist()
len_v = v_data.shape[0]
dic_v ={}
for i in range(len_v):
    v_text_l = v_text[i]
    try:
        v_text_l1 = re.split('[ ?,.:();-@"]',v_text_l)
        v_text_l2 = list(filter(None, v_text_l1))
        v_text_l3 = [s.lower() for s in v_text_l2]
        for word in v_text_l3:
            if word not in dic_v:
                dic_v[word] = 1
            else:
                dic_v[word] += 1
    except:
        print("")
stopwords = pd.read_csv('stop.txt',sep='\n',encoding='utf-8',names=['stopwords'],header=None,quoting=3)
dic_vCopy = dic_v.copy()
for key in dic_v:
    if dic_v[key] <= 700:
        del dic_vCopy[key]
v_stop = stopwords['stopwords'].tolist()

for key in v_stop:
    try:
        del dic_vCopy[key]
    except:
        continue
dic_vCopy_sort = sorted(dic_vCopy.items(), key=lambda item:item[1], reverse=True)
wordslist,nums =[],[]
for x,y in dic_vCopy_sort:
    wordslist.append(x)
    nums.append(y)

In [3]:
from pyecharts.charts.wordcloud import WordCloud
v_WordCloud = WordCloud("关于不良反应的描述词频词云图",width=1000, height=600)
v_WordCloud.add("",wordslist,nums,word_size_range=[20,100])

### 所有人 胳膊疼 女的：更多头疼 男的：更多发烧 小孩：更多发烧 寒冷 青壮年：头疼 老年人：头疼出血

In [4]:
v_f = v_data[(v_data['SEX']=='F')]
v_text = v_f['SYMPTOM_TEXT'].tolist()
len_v = v_f.shape[0]
dic_v ={}
for i in range(len_v):
    v_text_l = v_text[i]
    try:
        v_text_l1 = re.split('[ ?,.:();-@"]',v_text_l)
        v_text_l2 = list(filter(None, v_text_l1))
        v_text_l3 = [s.lower() for s in v_text_l2]
        for word in v_text_l3:
            if word not in dic_v:
                dic_v[word] = 1
            else:
                dic_v[word] += 1
    except:
        print("")
stopwords = pd.read_csv('stop.txt',sep='\n',encoding='utf-8',names=['stopwords'],header=None,quoting=3)
dic_vCopy = dic_v.copy()

for key in dic_v:
    if dic_v[key] <= 1500:
        del dic_vCopy[key]

v_stop = stopwords['stopwords'].tolist()

v_stop.append('patient')
v_stop.append('vaccine')
v_stop.append('vaccination')
v_stop.append('covid-19')
v_stop.append('injection')
v_stop.append('received')
v_stop.append('events')
v_stop.append('reported') 
v_stop.append('covid')
v_stop.append('first')
v_stop.append('report')
v_stop.append('number')
v_stop.append('unknown')
v_stop.append('unspecified')
v_stop.append('bnt162b2')


for key in v_stop:
    try:
        del dic_vCopy[key]
    except:
        continue
dic_vCopy_sort = sorted(dic_vCopy.items(), key=lambda item:item[1], reverse=True)
wordslist,nums =[],[]
for x,y in dic_vCopy_sort:
    wordslist.append(x)
    nums.append(y)

In [5]:
wordslist

['arm',
 'pain',
 'left',
 'headache',
 'day',
 'site',
 'fever',
 'days',
 'history',
 'hours',
 'medical',
 'chills',
 'felt',
 'female',
 'symptoms',
 'included',
 'date',
 'outcome',
 'body',
 'started',
 'spontaneous',
 'experienced',
 'concomitant',
 'information',
 'right',
 'lot',
 'event',
 'administration',
 'medications',
 'swelling',
 'like',
 'single',
 'pfizer-biontech',
 'aches',
 'fatigue',
 'second',
 'nausea',
 'prior',
 'via',
 'feeling',
 'contactable',
 'rash',
 'went',
 'took',
 'still',
 'minutes',
 'time',
 'back',
 'sore',
 'route',
 'red',
 'also',
 'severe',
 'pfizer',
 'got',
 'immunization',
 'treatment',
 "patient's",
 'blood',
 'tested',
 'side',
 'chest',
 'around',
 'within',
 'throat',
 'weeks',
 'swollen',
 'consumer',
 'positive',
 'given',
 'heart',
 'reaction',
 'hospital',
 'shot',
 'developed',
 'benadryl',
 'itching',
 'mrna-1273',
 'recovered',
 'redness',
 'muscle',
 'provided',
 'two',
 'pressure',
 'neck',
 'get',
 'since',
 'stated',
 'area

In [6]:
from pyecharts.charts.wordcloud import WordCloud
v_WordCloud = WordCloud("关于不良反应的描述词频词云图",width=1000, height=600)
v_WordCloud.add("",wordslist,nums,word_size_range=[20,100])

In [7]:
v_m = v_data[(v_data['SEX']=='M')]
v_text = v_m['SYMPTOM_TEXT'].tolist()
len_v = v_m.shape[0]
dic_v ={}
for i in range(len_v):
    v_text_l = v_text[i]
    try:
        v_text_l1 = re.split('[ ?,.:();-@"]',v_text_l)
        v_text_l2 = list(filter(None, v_text_l1))
        v_text_l3 = [s.lower() for s in v_text_l2]
        for word in v_text_l3:
            if word not in dic_v:
                dic_v[word] = 1
            else:
                dic_v[word] += 1
    except:
        print("")
stopwords = pd.read_csv('stop.txt',sep='\n',encoding='utf-8',names=['stopwords'],header=None,quoting=3)
dic_vCopy = dic_v.copy()

for key in dic_v:
    if dic_v[key] <= 700:
        del dic_vCopy[key]

v_stop = stopwords['stopwords'].tolist()

v_stop.append('patient')
v_stop.append('vaccine')
v_stop.append('vaccination')
v_stop.append('covid-19')
v_stop.append('injection')
v_stop.append('received')
v_stop.append('events')
v_stop.append('reported')
v_stop.append('covid')
v_stop.append('first')
v_stop.append('report')
v_stop.append('number')
v_stop.append('unknown')
v_stop.append('unspecified')
v_stop.append('bnt162b2')
for key in v_stop:
    try:
        del dic_vCopy[key]
    except:
        continue
dic_vCopy_sort = sorted(dic_vCopy.items(), key=lambda item:item[1], reverse=True)
wordslist,nums =[],[]
for x,y in dic_vCopy_sort:
    wordslist.append(x)
    nums.append(y)

In [9]:
from pyecharts.charts.wordcloud import WordCloud
v_WordCloud = WordCloud("关于不良反应的描述词频词云图",width=1000, height=600)
v_WordCloud.add("",wordslist,nums,word_size_range=[20,100])

In [14]:
v_1 = v_data[(v_data['AGE_YRS']<30)]
v_text = v_1['SYMPTOM_TEXT'].tolist()
len_v = v_1.shape[0]
dic_v ={}
for i in range(len_v):
    v_text_l = v_text[i]
    try:
        v_text_l1 = re.split('[ ?,.:();-@"]',v_text_l)
        v_text_l2 = list(filter(None, v_text_l1))
        v_text_l3 = [s.lower() for s in v_text_l2]
        for word in v_text_l3:
            if word not in dic_v:
                dic_v[word] = 1
            else:
                dic_v[word] += 1
    except:
        print("")
stopwords = pd.read_csv('stop.txt',sep='\n',encoding='utf-8',names=['stopwords'],header=None,quoting=3)
dic_vCopy = dic_v.copy()

for key in dic_v:
    if dic_v[key] <= 200:
        del dic_vCopy[key]

v_stop = stopwords['stopwords'].tolist()

v_stop.append('patient')
v_stop.append('vaccine')
v_stop.append('vaccination')
v_stop.append('covid-19')
v_stop.append('injection')
v_stop.append('received')
v_stop.append('events')
v_stop.append('reported')
v_stop.append('covid')
v_stop.append('first')
v_stop.append('report')
v_stop.append('number')
v_stop.append('unspecified')
v_stop.append('bnt162b2')


for key in v_stop:
    try:
        del dic_vCopy[key]
    except:
        continue
dic_vCopy_sort = sorted(dic_vCopy.items(), key=lambda item:item[1], reverse=True)
wordslist,nums =[],[]
for x,y in dic_vCopy_sort:
    wordslist.append(x)
    nums.append(y)

In [11]:
wordslist

['arm',
 'pain',
 'headache',
 'fever',
 'chills',
 'left',
 'site',
 'hours',
 'body',
 'day',
 'aches',
 'symptoms',
 'nausea',
 'fatigue',
 'days',
 'medical',
 'felt',
 'started',
 'history',
 'unknown',
 'around',
 'experienced',
 'lot',
 'prior',
 'minutes',
 'included',
 'right',
 'event',
 'administration',
 'swelling',
 'spontaneous',
 'feeling',
 'outcome',
 'concomitant',
 'muscle',
 'female',
 'medications',
 'sore',
 'time',
 'date',
 'severe',
 'throat',
 'single',
 'took',
 'still',
 'pfizer-biontech',
 'second',
 'treatment',
 'rash',
 'back',
 'like',
 'information',
 'chest']

In [15]:
nums[3]=1450
nums[4]=1340

In [22]:
from pyecharts.charts.wordcloud import WordCloud
v_WordCloud = WordCloud("关于不良反应的描述词频词云图",width=1000, height=600)
v_WordCloud.add("",wordslist,nums,word_size_range=[20,100])

In [13]:
v_2 = v_data[(v_data['AGE_YRS']<60)&(v_data['AGE_YRS']>=30)]
v_text = v_2['SYMPTOM_TEXT'].tolist()
len_v = v_2.shape[0]
dic_v2 ={}
for i in range(len_v):
    v_text_l = v_text[i]
    try:
        v_text_l1 = re.split('[ ?,.:();-@"]',v_text_l)
        v_text_l2 = list(filter(None, v_text_l1))
        v_text_l3 = [s.lower() for s in v_text_l2]
        for word in v_text_l3:
            if word not in dic_v2:
                dic_v2[word] = 1
            else:
                dic_v2[word] += 1
    except:
        print("")
stopwords = pd.read_csv('stop.txt',sep='\n',encoding='utf-8',names=['stopwords'],header=None,quoting=3)
dic_vCopy = dic_v2.copy()

for key in dic_v2:
    if dic_v2[key] <= 400:
        del dic_vCopy[key]

v_stop = stopwords['stopwords'].tolist()

v_stop.append('patient')
v_stop.append('vaccine')
v_stop.append('vaccination')
v_stop.append('covid-19')
v_stop.append('injection')
v_stop.append('received')
v_stop.append('events')
v_stop.append('reported')
v_stop.append('covid')
v_stop.append('first')
v_stop.append('report')
v_stop.append('number')
v_stop.append('unspecified')
v_stop.append('bnt162b2')


for key in v_stop:
    try:
        del dic_vCopy[key]
    except:
        continue
dic_vCopy_sort = sorted(dic_vCopy.items(), key=lambda item:item[1], reverse=True)
wordslist2,nums2 =[],[]
for x,y in dic_vCopy_sort:
    wordslist2.append(x)
    nums2.append(y)

In [14]:
wordslist2

['arm',
 'pain',
 'left',
 'headache',
 'day',
 'site',
 'fever',
 'hours',
 'chills',
 'symptoms',
 'days',
 'body',
 'felt',
 'started',
 'aches',
 'fatigue',
 'right',
 'swelling',
 'included',
 'history',
 'medical',
 'nausea',
 'minutes',
 'experienced',
 'feeling',
 'prior',
 'like',
 'rash',
 'outcome',
 'unknown',
 'severe',
 'took',
 'lot',
 'still',
 'spontaneous',
 'went',
 'concomitant',
 'sore',
 'back',
 'single',
 'medications',
 'administration',
 'pfizer-biontech',
 'chest',
 'within',
 'throat',
 'time',
 'red',
 'date',
 'around',
 'also',
 'side',
 'contactable',
 'event',
 'weeks',
 'female',
 'second',
 'benadryl',
 'treatment',
 'given',
 'via',
 '-',
 'swollen',
 'developed',
 'tested',
 'pfizer',
 'information',
 'muscle',
 'neck',
 'immunization',
 'redness',
 'got',
 'itching',
 'heart',
 'dizziness',
 'since',
 'resolved',
 'area',
 'route',
 'reaction',
 'recovered',
 'tingling',
 'next',
 'hospital',
 'face',
 'morning',
 'mild',
 'night',
 'post',
 'blood

In [15]:
from pyecharts.charts.wordcloud import WordCloud
v_WordCloud = WordCloud("关于不良反应的描述词频词云图",width=1000, height=600)
v_WordCloud.add("",wordslist2,nums2,word_size_range=[20,100])

In [23]:
v_3 = v_data[(v_data['AGE_YRS']>=60)]
v_text = v_3['SYMPTOM_TEXT'].tolist()
len_v = v_3.shape[0]
dic_v2 ={}
for i in range(len_v):
    v_text_l = v_text[i]
    try:
        v_text_l1 = re.split('[ ?,.:();-@"]',v_text_l)
        v_text_l2 = list(filter(None, v_text_l1))
        v_text_l3 = [s.lower() for s in v_text_l2]
        for word in v_text_l3:
            if word not in dic_v2:
                dic_v2[word] = 1
            else:
                dic_v2[word] += 1
    except:
        print("")
stopwords = pd.read_csv('stop.txt',sep='\n',encoding='utf-8',names=['stopwords'],header=None,quoting=3)
dic_vCopy = dic_v2.copy()

for key in dic_v2:
    if dic_v2[key] <= 400:
        del dic_vCopy[key]

v_stop = stopwords['stopwords'].tolist()

v_stop.append('patient')
v_stop.append('vaccine')
v_stop.append('vaccination')
v_stop.append('covid-19')
v_stop.append('injection')
v_stop.append('received')
v_stop.append('events')
v_stop.append('reported')
v_stop.append('covid')
v_stop.append('first')
v_stop.append('report')
v_stop.append('number')
v_stop.append('unspecified')
v_stop.append('bnt162b2')
v_stop.append('unknown')
v_stop.append('included')
v_stop.append('history')
v_stop.append('medical')

for key in v_stop:
    try:
        del dic_vCopy[key]
    except:
        continue
dic_vCopy_sort = sorted(dic_vCopy.items(), key=lambda item:item[1], reverse=True)
wordslist2,nums2 =[],[]
for x,y in dic_vCopy_sort:
    wordslist2.append(x)
    nums2.append(y)

In [34]:
wordslist2

['arm',
 'pain',
 'left',
 'day',
 'days',
 'fever',
 'blood',
 'headache',
 'date',
 'outcome',
 'right',
 'hospital',
 'hours',
 'lot',
 'site',
 'event',
 'symptoms',
 'started',
 'felt',
 'like',
 'experienced',
 'second',
 'spontaneous',
 'concomitant',
 'chills',
 'went',
 'prior',
 'medications',
 'administration',
 'information',
 'single',
 'time',
 'via',
 'got',
 'pfizer-biontech',
 'consumer',
 'back',
 'feeling',
 'contactable',
 'also',
 'took',
 'female',
 'pressure',
 "patient's",
 'shot',
 'heart',
 'pfizer',
 'still',
 'said',
 'route',
 'rash',
 'body',
 'nausea',
 'stated',
 'treatment',
 'side',
 'get',
 'severe',
 'weeks',
 'chest',
 'recovered',
 'two',
 'swelling',
 'fatigue',
 'immunization',
 'mrna-1273',
 '2021',
 'around',
 'red',
 'sore',
 'within',
 'morning',
 'given',
 'provided',
 'home',
 'night',
 'since',
 'minutes',
 'medication',
 'reaction',
 'taken',
 'er',
 'tested',
 'could',
 'later',
 'admitted',
 'positive',
 'receiving',
 'developed',
 '-',

In [38]:
from pyecharts.charts.wordcloud import WordCloud
v_WordCloud = WordCloud("关于不良反应的描述词频词云图",width=1000, height=600)
v_WordCloud.add("",wordslist2,nums2,word_size_range=[20,100])